# A Behavioral-Economic Demand Analysis of Social Reinforcement
### A Python Replication of Schulingkamp et al. (2023), *Frontiers in Psychology*

---

### Project Objective
This notebook provides a complete, reproducible Python workflow for the analyses presented in the following publication:

> Schulingkamp, R., Wan, H., & Hackenberg, T. D. (2023). Social familiarity and reinforcement value: a behavioral-economic analysis of demand for social interaction with cagemate and non-cagemate female rats. *Frontiers in Psychology*, *14*, 1158365. https://doi.org/10.3389/fpsyg.2023.1158365

The study's goal is to quantify the reinforcing value of social interaction in rats using a behavioral-economic demand analysis. Specifically, it tests how this value is affected by **social familiarity** and **reinforcer magnitude** (duration).

This notebook demonstrates two distinct analytical approaches to fitting the **Zero-Bounded Exponential (ZBEn) demand model** to the data:
1.  A **frequentist, individual-level analysis** using `lmfit` for nonlinear least-squares.
2.  A **Bayesian multilevel analysis** using `pymc` to account for the repeated-measures data structure from a small sample.

The data for this study is available in the Supplementary Material of the original publication at the [publisher's website](https://www.frontiersin.org/articles/10.3389/fpsyg.2023.1158365/full#supplementary-material).

### Analysis Workflow
1.  **Setup & Data Processing**: Loads Python libraries, defines helper functions, and processes the raw data into an analysis-ready format with dummy variables.
2.  **Frequentist Analysis (Individual-Level)**: Fits the ZBEn demand model to each rat's data separately and performs linear contrasts on the estimated parameters.
3.  **Bayesian Analysis (Multilevel)**: Fits a single, comprehensive Bayesian nonlinear mixed-effects model to the entire dataset, treating individual rats as random effects.

In [ ]:
# --- Environment Setup ---
#
# This cell installs the required Python packages for the analysis.
# Uncomment and run this cell only if you are setting up a new environment.

# import sys
# !{sys.executable} -m pip install pandas numpy scipy lmfit pymc arviz

In [ ]:
# --- 1. SETUP: IMPORTS, FUNCTIONS, AND DATA PROCESSING ---

# --- 1.1 Load Libraries ---

import pandas as pd
import numpy as np
import warnings
from lmfit import Model
import pymc as pm
import arviz as az
import pytensor.tensor as pt

# Suppress warnings for a cleaner final report
warnings.filterwarnings('ignore')
# Set pandas display options for consistent formatting
pd.options.display.float_format = '{:.3f}'.format


# --- 1.2 Helper Functions for ZBEn Model ---

def transform_IHS(x):
    """
    Inverse Hyperbolic Sine (IHS) transformation.
    A log-like transformation that is defined at zero, required by the ZBEn model.
    """
    return np.log10(0.5 * x + np.sqrt(0.25 * (x**2) + 1))


# --- 1.3 Load and Process Data ---

# Load the raw data from a CSV file
raw_demand_df = pd.read_csv("dat.csv")
print("Raw data loaded successfully.")

# --- Process data into an analysis-ready format ---
processed_df = raw_demand_df.copy()
processed_df['active_session_hours'] = (processed_df['active_ses_time'] / 60) / 60
processed_df['interaction_rate'] = processed_df['interact'] / processed_df['active_session_hours']
processed_df['lq'] = transform_IHS(processed_df['interaction_rate'])
processed_df['familiarity_label'] = np.where(processed_df['familiarity'] == 1, "Cagemate", "Non_cagemate")
processed_df['duration_label'] = processed_df['cond'].replace({
    "10sec": "10_Sec", "30sec": "30_Sec", "60sec": "60_Sec"
})

# Aggregate data by averaging across sessions with the same price (fr)
aggregated_df = processed_df.groupby(['pair', 'familiarity_label', 'duration_label', 'fr'], as_index=False).agg(
    interaction_rate=('interaction_rate', 'mean'),
    lq=('lq', 'mean')
)

# --- Create dummy variables for regression models ---
# First, create a single interaction-style column for all 6 conditions
aggregated_df['condition'] = (
    aggregated_df['familiarity_label'] + "_" + aggregated_df['duration_label']
)

# Use pd.get_dummies to create the binary predictor variables
model_ready_df = pd.get_dummies(aggregated_df, columns=['condition'], prefix='', prefix_sep='')

# Rename columns to be short and valid variable names for model formulas
model_ready_df = model_ready_df.rename(columns={
    'Cagemate_10_Sec': 'f1',
    'Cagemate_30_Sec': 'f3',
    'Cagemate_60_Sec': 'f6',
    'Non_cagemate_10_Sec': 'u1',
    'Non_cagemate_30_Sec': 'u3',
    'Non_cagemate_60_Sec': 'u6'
})

print("Individual-level data processed and ready for analysis.")

Raw data loaded successfully.
Individual-level data processed and ready for analysis.


,pair,familiarity_label,duration_label,fr,interaction_rate,lq,f1,f3,f6,u1,u3,u6
0,1,Cagemate,10_Sec,1,19.099,1.234,True,False,False,False,False,False
1,1,Cagemate,10_Sec,2,6.136,0.799,True,False,False,False,False,False
2,1,Cagemate,10_Sec,5,2.017,0.385,True,False,False,False,False,False
3,1,Cagemate,10_Sec,10,0.000,0.000,True,False,False,False,False,False
4,1,Cagemate,30_Sec,1,24.800,1.381,False,True,False,False,False,False


## 2. Frequentist Analysis (Individual-Level)

This section replicates the first analytical approach from the paper, fitting the **Zero-Bounded Exponential (ZBEn) demand model** to each rat's data separately for all experimental conditions. This individual-level method allows for a direct examination of between-subject variability.

The ZBEn model is a nonlinear function that describes how consumption of a reinforcer (here, social interaction rate) decreases as its price increases. By fitting this model using the `lmfit` library, we can extract two key parameters that quantify the value of the reinforcer for each condition:
-   **$Q_{0}$ (Demand Intensity)**: The predicted level of consumption when the price is zero. A higher $Q_{0}$ indicates a higher overall demand.
-   **$\alpha$ (Elasticity)**: The rate at which consumption decreases as the price increases. A higher $\alpha$ indicates that demand is more sensitive to price (i.e., more elastic).

After fitting the models, we perform linear contrasts on the estimated parameters to test the primary hypotheses: whether social familiarity or interaction duration systematically affected demand intensity ($Q_{0}$) or elasticity ($\alpha$).

In [ ]:
from lmfit import Model
from scipy.stats import t

# --- 2.1 Define and Fit the ZBEn Demand Model for Each Rat ---

def zbe_model_func(fr, f1, f3, f6, u1, u3, u6, 
                   af1, af3, af6, au1, au3, au6, 
                   qf1, qf3, qf6, qu1, qu3, qu6):
    """
    Zero-Bounded Exponential (ZBEn) demand model function for lmfit.
    This function uses dummy variables (f1, u1, etc.) to estimate a unique
    alpha (a) and Q0 (q) for each of the 6 experimental conditions.
    """
    alpha_term = np.exp(af1*f1 + af3*f3 + af6*f6 + au1*u1 + au3*u3 + au6*u6)
    q0_term = qf1*f1 + qf3*f3 + qf6*f6 + qu1*u1 + qu3*u3 + qu6*u6
    lhs_q0 = transform_IHS(q0_term)
    return lhs_q0 * np.exp((-alpha_term / lhs_q0) * q0_term * fr)

# Create an lmfit Model object
zbe_lmfit_model = Model(
    zbe_model_func, 
    independent_vars=['fr', 'f1', 'f3', 'f6', 'u1', 'u3', 'u6']
)

# --- Loop through each rat to fit the model and perform contrasts ---
parameter_list = []
contrast_list = []
for rat_id in model_ready_df['pair'].unique():
    
    rat_data = model_ready_df[model_ready_df['pair'] == rat_id]
    
    # Set up initial parameter values for the fit
    fit_params = zbe_lmfit_model.make_params(
        af1=-6, af3=-6, af6=-6, au1=-6, au3=-6, au6=-6,
        qf1=50, qf3=50, qf6=50, qu1=50, qu3=50, qu6=50
    )
    # Constrain Q0 parameters to be non-negative
    for p_name in fit_params:
        if p_name.startswith('q'):
            fit_params[p_name].set(min=0)
            
    # Define the independent variables for the fit
    independent_vars = {
        'fr': rat_data['fr'], 'f1': rat_data['f1'], 'f3': rat_data['f3'],
        'f6': rat_data['f6'], 'u1': rat_data['u1'], 'u3': rat_data['u3'],
        'u6': rat_data['u6']
    }
    
    # Fit the model
    result = zbe_lmfit_model.fit(rat_data['lq'], fit_params, **independent_vars)
    
    # Store the parameter estimates
    params_df = pd.DataFrame.from_dict(result.params.valuesdict(), orient='index', columns=['estimate'])
    params_df['rat_id'] = rat_id
    parameter_list.append(params_df)

    # --- Perform Linear Contrasts Manually ---
    contrasts_to_test = {
        'alpha': {'af1':1, 'af3':1, 'af6':1, 'au1':-1, 'au3':-1, 'au6':-1},
        'Q0':    {'qf1':1, 'qf3':1, 'qf6':1, 'qu1':-1, 'qu3':-1, 'qu6':-1}
    }
    
    # Get the covariance matrix and ensure parameter order is correct
    covar_matrix = result.covar
    param_names = result.var_names

    for param_name, expression in contrasts_to_test.items():
        # Create a contrast vector C with weights, ordered to match the covariance matrix
        contrast_vector = np.array([expression.get(p, 0) for p in param_names])
        
        # Calculate the contrast estimate: C * B
        est = contrast_vector @ np.array([result.params[p].value for p in param_names])
        
        # Calculate the standard error of the contrast: sqrt(C' * Cov(B) * C)
        variance = contrast_vector.T @ covar_matrix @ contrast_vector
        stderr = np.sqrt(variance)
        
        # Calculate t-statistic and two-tailed p-value
        t_stat = est / stderr if stderr > 0 else 0
        p_val = t.sf(np.abs(t_stat), df=result.nfree) * 2
        
        contrast_list.append({
            'rat_id': rat_id, 'parameter': param_name, 'contrast': 'Cagemate vs. Non-cagemate',
            'estimate': est, 'std_error': stderr, 't_value': t_stat, 'p_value': p_val
        })

# --- 2.2 Format and Display Results ---

# Combine parameter estimates from all rats into a single DataFrame
individual_parameter_estimates = pd.concat(parameter_list).reset_index().rename(columns={'index': 'term'})
individual_parameter_estimates[['parameter_type', 'familiarity_code', 'duration_code']] = \
    individual_parameter_estimates['term'].str.extract(r'(a|q)(f|u)(\d)').fillna('')
individual_parameter_estimates['parameter'] = np.where(individual_parameter_estimates['parameter_type'] == 'a', 'alpha', 'Q0')
individual_parameter_estimates['Familiarity'] = np.where(individual_parameter_estimates['familiarity_code'] == 'f', 'Cagemate', 'Non-cagemate')
individual_parameter_estimates['Duration'] = individual_parameter_estimates['duration_code'].replace({'1':'10 Sec', '3':'30 Sec', '6':'60 Sec'})
final_params_df = individual_parameter_estimates[['rat_id', 'Familiarity', 'Duration', 'parameter', 'estimate']]

# Combine contrast results into a single DataFrame
final_contrasts_df = pd.DataFrame(contrast_list)

print("--- Estimated ZBEn Parameters (Individual Level) ---")
display(final_params_df)
print("\n--- Linear Contrast Results ---")
display(final_contrasts_df)

## 3. Bayesian Nonlinear Multilevel Analysis

This section presents a more sophisticated analytical approach by fitting a single **Bayesian nonlinear multilevel model** to the entire dataset using `PyMC`. This method offers several advantages over the individual-level frequentist analysis:

-   **Principled Regularization**: By treating individual rats as "random effects," the model uses a technique called **partial pooling**. This allows each rat's parameter estimates to be informed by the overall group average, leading to more stable and realistic estimates, especially with small sample sizes.
-   **Unified Framework**: Instead of fitting 24 separate models (4 rats x 6 conditions), we fit one comprehensive model. This provides a single, coherent framework for testing hypotheses about the main effects of familiarity and duration.
-   **Full Posterior Inference**: This approach yields the full posterior distribution for every parameter and contrast, giving us a complete picture of our uncertainty about the effects.

The model estimates fixed effects for the experimental conditions (familiarity and duration) and random effects for each rat's deviation from those fixed effects, directly modeling the between-subject variability within a hierarchical structure.

In [19]:
# --- 3.1 Define and Fit the Bayesian Multilevel Model ---

print("--- Building and Fitting Bayesian Model with PyMC (this may take several minutes) ---")

# --- Use aggregated_df for the Bayesian model, as it retains the 'condition' column ---
bayes_df = aggregated_df.copy()

# Define coordinates for the model dimensions, a best practice for PyMC
condition_names = sorted(bayes_df['condition'].unique())
rat_names = sorted(bayes_df['pair'].unique())
coords = {
    "condition": condition_names,
    "rat": rat_names,
    "obs_id": bayes_df.index
}

# Create numeric indices for mapping observations to the correct parameters
rat_idx = pd.Categorical(bayes_df['pair'], categories=rat_names).codes
condition_idx = pd.Categorical(bayes_df['condition'], categories=condition_names).codes

with pm.Model(coords=coords) as multilevel_zbe_model:
    # --- Priors for Random Effects (Rat-level variability) ---
    # Non-centered parameterization for better sampling efficiency
    sd_alpha = pm.HalfCauchy("sd_alpha", beta=2.5)
    z_alpha = pm.Normal("z_alpha", mu=0, sigma=1, dims="rat")
    rat_effect_alpha = pm.Deterministic("rat_effect_alpha", z_alpha * sd_alpha, dims="rat")
    
    sd_q0 = pm.HalfCauchy("sd_q0", beta=2.5)
    z_q0 = pm.Normal("z_q0", mu=0, sigma=1, dims="rat")
    rat_effect_q0 = pm.Deterministic("rat_effect_q0", z_q0 * sd_q0, dims="rat")

    # --- Priors for Fixed Effects (Condition-level means) ---
    # Weakly informative priors based on the frequentist results and theory
    # Parameters are estimated on the log scale to ensure they are positive
    log_alpha_coeffs = pm.Normal("log_alpha", mu=-6, sigma=2.5, dims="condition")
    log_q0_coeffs = pm.Normal("log_q0", mu=4, sigma=2.5, dims="condition")

    # --- Combine Fixed and Random Effects ---
    log_alpha_est = log_alpha_coeffs[condition_idx] + rat_effect_alpha[rat_idx]
    log_q0_est = log_q0_coeffs[condition_idx] + rat_effect_q0[rat_idx]
    
    alpha = pm.Deterministic("alpha", pt.exp(log_alpha_est), dims="obs_id")
    q0 = pm.Deterministic("q0", pt.exp(log_q0_est), dims="obs_id")

    # --- ZBEn Model Equation (Likelihood Mu) ---
    ihs_q0 = transform_IHS(q0)
    mu = ihs_q0 * pt.exp((-alpha / ihs_q0) * q0 * bayes_df['fr'].values)

    # --- Likelihood ---
    sigma = pm.HalfCauchy("sigma", beta=2.5)
    lq_obs = pm.Normal("lq_obs", mu=mu, sigma=sigma, observed=bayes_df['lq'], dims="obs_id")
    
    # --- MCMC Sampling ---
    idata = pm.sample(draws=2000, tune=2000, chains=4, cores=4, 
                      target_accept=0.95, progressbar=False)


# --- 3.2 Analyze Posterior Contrasts ---

def summarize_contrasts(posterior_draws):
    """
    Helper function to calculate median and 95% HDI for contrast distributions.
    This version is corrected to use xarray syntax before converting to pandas.
    """
    # Calculate quantiles using xarray's method, which creates a 'quantile' coordinate
    quantiles = posterior_draws.quantile([0.025, 0.5, 0.975], dim=("chain", "draw"))
    # Re-label the values of the 'quantile' coordinate
    quantiles = quantiles.assign_coords(quantile=['lower_ci', 'median', 'upper_ci'])
    # Convert the labeled xarray object to a pandas Series
    return quantiles.to_series()

# --- Define condition groups for contrasts ---
cagemate_conds = [c for c in condition_names if 'Cagemate' in c]
non_cagemate_conds = [c for c in condition_names if 'Non_cagemate' in c]
conds_10_sec = [c for c in condition_names if '10_Sec' in c]
conds_60_sec = [c for c in condition_names if '60_Sec' in c]

# --- Extract posterior draws for alpha and Q0 ---
posterior_log_alpha = idata.posterior["log_alpha"]
posterior_log_q0 = idata.posterior["log_q0"]

# --- Calculate Alpha Contrasts ---
contrast_alpha_familiarity = (
    posterior_log_alpha.sel(condition=cagemate_conds).mean(dim="condition") -
    posterior_log_alpha.sel(condition=non_cagemate_conds).mean(dim="condition")
)
contrast_alpha_duration = (
    posterior_log_alpha.sel(condition=conds_60_sec).mean(dim="condition") -
    posterior_log_alpha.sel(condition=conds_10_sec).mean(dim="condition")
)

# --- Calculate Q0 Contrasts ---
contrast_q0_familiarity = (
    posterior_log_q0.sel(condition=cagemate_conds).mean(dim="condition") -
    posterior_log_q0.sel(condition=non_cagemate_conds).mean(dim="condition")
)
contrast_q0_duration = (
    posterior_log_q0.sel(condition=conds_60_sec).mean(dim="condition") -
    posterior_log_q0.sel(condition=conds_10_sec).mean(dim="condition")
)

# --- Format and Display Results ---
alpha_contrasts = pd.DataFrame({
    'Familiarity (Cagemate - Non)': summarize_contrasts(contrast_alpha_familiarity),
    'Duration (60s - 10s)': summarize_contrasts(contrast_alpha_duration)
})

q0_contrasts = pd.DataFrame({
    'Familiarity (Cagemate - Non)': summarize_contrasts(contrast_q0_familiarity),
    'Duration (60s - 10s)': summarize_contrasts(contrast_q0_duration)
})

print("\n\n--- Posterior Summary for Contrasts on log(alpha) ---")
display(alpha_contrasts.T)

print("\n--- Posterior Summary for Contrasts on log(Q0) ---")
display(q0_contrasts.T)

--- Building and Fitting Bayesian Model with PyMC (this may take several minutes) ---


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sd_alpha, z_alpha, sd_q0, z_q0, log_alpha, log_q0, sigma]
Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 23 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.




--- Posterior Summary for Contrasts on log(alpha) ---


quantile,lower_ci,median,upper_ci
Familiarity (Cagemate - Non),0.244,0.480,0.723
Duration (60s - 10s),-0.076,0.214,0.496



--- Posterior Summary for Contrasts on log(Q0) ---


quantile,lower_ci,median,upper_ci
Familiarity (Cagemate - Non),-0.873,-0.467,-0.062
Duration (60s - 10s),-0.644,-0.170,0.305
